In [1]:
!pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## **Two of the final versions of Lab2 code are here which were somehow able to give the same output as the one shown on the pdf file.**



### **Here counter class is called once and its instance acquires preemptiveResource with 20 slots each representing individual counters. When new counter needs opening, it simply moves to next slot and counter open message is printed but counter class is not accessed anymore.**

In [2]:
import random
import simpy

RANDOM_SEED = 42  
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
NO_OF_COUNTERS = 0     #variable that keeps track of number of counters
COUNT_CUSTOMER = 0     #variable that keeps track of number of customers


'''The counter class that opens a new counter object. Only 1 counter object is called which acquires PreemptiveResource 
only once with capacity 20 which acts as 20 counters and will take care of coming customers with preemption'''
class Counter(object):
  
    def __init__(self, env):       #Constructor
        global NO_OF_COUNTERS       #declared global before using global variables orelse python will treat it as local variable
        NO_OF_COUNTERS += 1         #increased by 1 for each object calls. Since called once NO_OF_COUNTERS becomes 1 from 0

        self.env = env              #this particular class instance's local env (environment) = assigned env passed in param
        self.hold_resource = simpy.PreemptiveResource( self.env, capacity = 20 )  #PreemptiveResource with capacity 20 for our environment called from simpy

        print( '%7.4f Counter%02d: Opened' % (self.env.now, NO_OF_COUNTERS) )     #prints counter open message at the beginning


'''a method that will generate customer randomly within the interval of (1/INTERVAL_CUSTOMERS) seconds and 
also tracks if more than 5 customers are waiting in line and opens new counter if they are'''
def generate_customer(env, number, interval, counter):    

    global COUNT_CUSTOMER
    global NO_OF_COUNTERS

    for i in range(number):        #loops upto NEW_CUSTOMERS (total number of customers allowed in this simulation)
        COUNT_CUSTOMER += 1        #update the customer count by 1 -> creating each new customer with each loop iteration

        if (COUNT_CUSTOMER > 5):    #Condition checks -> if more than 5 customers waiting in line unattended as when each 
                                    #customer is tend to, they leave decreasing customer count by 1 
          NO_OF_COUNTERS += 1       #increases number of counters by 1 indicating new counter has been opened

          print("%7.4f: More than 5 customers waiting in line" % env.now)
          print( '%7.4f Counter%02d: Opened' % ( env.now, NO_OF_COUNTERS ) )  #prints environment time right now (0.0000 format) & the opened counter number (00 format)
                                                                              
          COUNT_CUSTOMER = 1     #COUNT_CUSTOMER set to 1 because when new counter opens we have to again keep track if this phenomena repeats in the future and handle it

        c = Customer(env, COUNT_CUSTOMER, 'Customer%02d' % i, counter, time_in_bank=12.0)   #a instance/ object of Customer class created -> indicates creation of each customer 

        t = random.expovariate(1.0 / interval)    #selects a random value from the interval (1/INTERVAL_CUSTOMERS) -> (1/10.0) here
        yield env.timeout(t)             #env times out after 't' time -> after 't' time another customer creation process starts


'''Customer class -> here customer handles business in avaible counters (in customer() method)'''
class Customer(object):
  
  def __init__(self, env, count_cus, name, counter, time_in_bank):

        self.env = env                   #env, COUNT_CUSTOMER, customer number, instance of counter and time to be spent in bank 
        self.count_cus = count_cus       #all passed to each customer instance as "self.___" from the respective param initializations 
        self.name = name
        self.counter = counter
        self.time_in_bank = time_in_bank        #a considered value for Time to be spend in bank for a customer to complete business generally  
        self.process = env.process(self.customer())     #Starting process -> calling customer() method -> here for this particular instance 


  '''This method handles all the tasks from when a customer arrives to which counter tends to
  them to new counter opening when customer loses patience to when they finally renege'''
  def customer(self):
 
      global NO_OF_COUNTERS

      arrive = self.env.now    #when customer arrives at bank -> stores environment time at this moment      
      print( '%7.4f %s: Here I am' % (arrive, self.name) )   #prints message of customers arrival with arrive time and customer name

      with self.counter.hold_resource.request() as req:      #Request access on the resource. Event triggered once access is granted

        patience = random.uniform( MIN_PATIENCE, MAX_PATIENCE )    #patience of each customer is determined by randomly taking a float value 
                                                                   #from the Interval ( MIN_PATIENCE, MAX_PATIENCE )        
        yield req       # Wait for the counter resource access

        # print( "..............%7.4f patience %7.4f" % (self.env.now, patience) )
        # env.process( self.processing(req, patience) )

        wait = self.env.now - arrive      #time for which customer is waiting for the counter 


        if ( len(self.counter.hold_resource.users) <= NO_OF_COUNTERS ):     #checks if length of user list for this resource is < or = to total number of counters-> as when 1 
                                                                      #capacity slot is taken, moves to next slot which means next counter. True means hasn't moved to next counter

            print('%7.4f %s: Waited %6.3f' % (self.env.now, self.name, wait))      #prints how long customer waited along with env time now

            tib = random.expovariate(1.0 / self.time_in_bank)     #time required by this customer to do buisness in bank counter -> randomly from interval (1.0/time_in_bank)
            yield self.env.timeout(tib)          #enviroment times out after 'tib' time has passed starting from time now

            # j = 0
            # for i in range(len(self.counter.hold_resource.users)):    #Here i was running a loop through the userlist of counter to match to req 
            #   if (self.counter.hold_resource.users[i] == req):        #if any slot matched that slot would be recorded in j and +1 as loop starts from 0
            #     j = i + 1
            # print( '%7.4f %s: Served by Counter %02d' % (env.now, name, j) )  #This print statement made more sense as j was being printed as counter number,  
                                                                      #went with 2nd one to match the pdf. Although 'tib' time in pdf is different to mine so j is accurate here 

            print( '%7.4f %s: Served by Counter %02d' % (self.env.now, self.name, self.counter.hold_resource.count) )  #prints which counter served this customer and time now
                                                                      #user finishes, count of users currently using the resource decreases indicating free counter in this slot 
            
            # print( '**********************', len(self.counter.hold_resource.queue) )
            # print( '**********************', len(self.counter.hold_resource.users) )
            # print( '**********************', NO_OF_COUNTERS )

            self.count_cus -= 1        #deceasing number of customer count as customer finished business and leaves
            

        elif ( len(self.counter.hold_resource.users) > NO_OF_COUNTERS  and  NO_OF_COUNTERS <= 20 ): #if length of userlist > number of counters & counters are less than 20, meaning
                                                                                                    #user has no counter free and moved to next capacity slot -> we open new counter

            yield self.env.timeout(patience)    #env times out after patience has reached -> time to open new counter before customer leaves
            print( '%7.4f %s: About to give up after %6.3f' % (self.env.now, self.name, patience) )    #prints when customer about to giveup and extent of their patience

            NO_OF_COUNTERS += 1       #increasing number of counter indicating new counter opened

            print( '%7.4f Counter%02d: Opened' % (self.env.now, NO_OF_COUNTERS) )   #printing this event

            tib = random.expovariate(1.0 / self.time_in_bank)
            yield self.env.timeout(tib)
            
            # j = 0
            # for i in range(len(self.counter.hold_resource.users)):
            #   if (self.counter.hold_resource.users[i] == req):
            #     j = i + 1
            # print( '%7.4f %s: Served by Counter %02d' % (self.env.now, self.name, j) )

            print( '%7.4f %s: Served by Counter %02d' % (self.env.now, self.name, self.counter.hold_resource.count) )
            self.count_cus -= 1


        else:    #None of previous conditions meet -> means all counters occupied & total counters have reached limit(20) so new counter can't be opened, customer reneges

            print('%7.4f %s: RENEGED' % (self.env.now, self.name))      #prints customer reneges along with customer name and time of renege  
            yield self.env.timeout(patience)          #env timed out as customer reaches patience limit
            self.count_cus -= 1


      global COUNT_CUSTOMER          #global COUNT_CUSTOMER value is updated with local self.count_cus tracking till now, to keep track globally
      COUNT_CUSTOMER = self.count_cus


  # def processing (self, req, patience):
  #     if ( len(self.counter.hold_resource.users) > NO_OF_COUNTERS  and  NO_OF_COUNTERS <= 20 ):


# Setup and start the simulation
print('Bank renege modified')
random.seed(RANDOM_SEED)
env = simpy.Environment()     #environment called from simpy and initialized
counter = Counter(env)        #an instance of Counter class opened and env passed on to that instance
# Start processes and run
env.process( generate_customer( env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter ) )     #generate_customer() process started 
env.run()         #environment process run


Bank renege modified
 0.0000 Counter01: Opened
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Served by Counter 01
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
13.9003 Customer02: About to give up after  1.174
13.9003 Counter02: Opened
20.4769 Customer02: Served by Counter 02
23.7507 Customer01: Served by Counter 01
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
35.3018 Customer04: Here I am
35.3018 Customer04: Waited  0.000
43.4463 Customer03: Served by Counter 02
47.8958 Customer04: Served by Counter 01


### **Here we use two nested preemptive resources. 'new_counter' is used when new counter is opened...instance of Counter class opens each new counter and is called multiple times when new counter needs opening, and each of them has capacity 1 and counter_manager (another preemptive Resource) manages new_counters with 20 capacity slots with preemption. Each slot is there for manging each of 20 counters**

In [3]:
'''Here the same thing happens but with two nested preemptive resources. 'new_counter' is used when new counter is opened 
and each of them has capacity 1 and counter_manager manages new_counters with 20 capacity slots with preemption'''

import random
import simpy
RANDOM_SEED = 42  
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
NO_OF_COUNTERS = 0     #variable that keeps track of number of counters
COUNT_CUSTOMER = 0     #variable that keeps track of number of customers


class Counter(object):
    # instances = {}
    def __init__(self, env):       
        global NO_OF_COUNTERS       
        NO_OF_COUNTERS += 1         
        self.env = env             
        self.new_counter = simpy.PreemptiveResource( self.env, capacity = 1 ) 
        print( '%7.4f Counter%02d: Opened' % (self.env.now, NO_OF_COUNTERS) )     
        # Counter.instances["Counter %02d" % self.counter_name] = self 


def generate_customer(env, number, interval, counter_manager, counter ):    
    global COUNT_CUSTOMER
    global NO_OF_COUNTERS

    for i in range(number):        
        COUNT_CUSTOMER += 1       
        if (COUNT_CUSTOMER > 5):     
          print("%7.4f: More than 5 customers waiting in line" % env.now)
          counter = Counter(env)                                                                               
          COUNT_CUSTOMER = 1    

        c = customer(env, COUNT_CUSTOMER, 'Customer%02d' % i, counter_manager, counter, time_in_bank=12.0 ) 
        env.process(c)
        t = random.expovariate(1.0 / interval)    
        yield env.timeout(t)             


def customer(env, count_cus, name, counter_manager, counter, time_in_bank ):
    global NO_OF_COUNTERS
    arrive = env.now    
    print( '%7.4f %s: Here I am' % (arrive, name) )

    with counter_manager.request() as req:   #counter_manager manages counter.new_counter so first we yield its request, when one counter is busy they move to next
      yield req                              #each of 20 slots of counter_manager represents 20 individual counters and preemption will give customers the free one first

      with counter.new_counter.request() as req2:                #next we yield counter.new_counter's request, here customers will wait for a counter
          patience = random.uniform( MIN_PATIENCE, MAX_PATIENCE )
          results = yield req2 | env.timeout(patience)         

          wait = env.now - arrive


          if (req2 in results):                                   #checking if the counter they are waiting in is available
              print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
              tib = random.expovariate(1.0 / time_in_bank)
              yield env.timeout(tib)

              # j = 0
              # for i in range(len(counter_manager.users)):
              #   if (counter_manager.users[i] == req):
              #     j = i + 1              
              # print( '%7.4f %s: Served by Counter %02d' % (env.now, name, j) )
              
              print( '%7.4f %s: Served by Counter %02d' % (env.now, name, counter_manager.count) )            
              count_cus -= 1


          elif (req2 not in results and len(counter_manager.users) <= NO_OF_COUNTERS):   #since their first option was not available, 
              print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))                    #they move to other available counters
              tib = random.expovariate(1.0 / time_in_bank)
              yield env.timeout(tib)

              # j = 0
              # for i in range(len(counter_manager.users)):
              #   if (counter_manager.users[i] == req):
              #     j = i + 1              
              # print( '%7.4f %s: Served by Counter %02d' % (env.now, name, j) )
              
              print( '%7.4f %s: Served by Counter %02d' % (env.now, name, counter_manager.count) )            
              count_cus -= 1


          elif ( req2 not in results  and  len(counter_manager.users) > NO_OF_COUNTERS  and  NO_OF_COUNTERS <= 20 ):    #none of the other counters are available 
              print( '%7.4f %s: About to give up after %6.3f' % (env.now, name, patience) )                             #so new counter opened

              counter = Counter(env)
              tib = random.expovariate(1.0 / time_in_bank)
              yield env.timeout(tib)

              # j = 0
              # for i in range(len(counter_manager.users)):
              #   if (counter_manager.users[i] == req):
              #     j = i + 1              
              # print( '%7.4f %s: Served by Counter %02d' % (env.now, name, j) )

              print( '%7.4f %s: Served by Counter %02d' % (env.now, name, counter_manager.count) )
              count_cus -= 1

          else:                        #all 20 counters occupied, new counter can't be opened so they renege when patience gone
              #We reneged
              print('%7.4f %s: RENEGED' % (env.now, name))
              yield env.timeout(patience)
              count_cus -= 1

      global COUNT_CUSTOMER
      COUNT_CUSTOMER = count_cus


print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()
counter = Counter(env)

counter_manager = simpy.PreemptiveResource( env, capacity = 20 )   #managers counters in 20 slots -> another preeptive resource with 20 capacity 

env.process( generate_customer( env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter_manager, counter ) )
env.run()

Bank renege
 0.0000 Counter01: Opened
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Served by Counter 01
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
13.9003 Customer02: About to give up after  1.174
13.9003 Counter02: Opened
20.4769 Customer02: Served by Counter 02
23.7507 Customer01: Served by Counter 01
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
35.3018 Customer04: Here I am
36.6995 Customer04: Waited  1.398
43.4463 Customer03: Served by Counter 02
49.2934 Customer04: Served by Counter 01
